In [ ]:
# 🎯 Test Loss: 0.2881 | Test Accuracy: 0.9395
# Test Loss: 0.2986 | Test Accuracy: 0.9107
#🎯 Test Loss: 0.2297 | Test Accuracy: 0.9253
# 🎯 Test Loss: 0.2906 | Test Accuracy: 0.8949


In [ ]:
# vit模型，添加复合注意力结构CBAM + ECA（Efficient Channel Attention）

In [7]:
# ✅ 1. 导入库
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models.vision_transformer import vit_b_16, ViT_B_16_Weights
from tqdm.notebook import tqdm

# ✅ 2. 忽略隐藏目录的数据集类
class FilteredImageFolder(ImageFolder):
    def find_classes(self, directory):
        classes = [d.name for d in os.scandir(directory) if d.is_dir() and not d.name.startswith('.')]
        classes.sort()
        class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}
        return classes, class_to_idx

# ✅ 3. 定义 CBAM + ECA 模块
class ECAAttention(nn.Module):
    def __init__(self, channels, k_size=3):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv = nn.Conv1d(1, 1, kernel_size=k_size, padding=(k_size-1)//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        y = self.avg_pool(x)
        y = self.conv(y.squeeze(-1).transpose(-1, -2))
        y = self.sigmoid(y).transpose(-1, -2).unsqueeze(-1)
        return x * y.expand_as(x)

class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=8):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.shared_MLP = nn.Sequential(
            nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg = self.shared_MLP(self.avg_pool(x))
        max = self.shared_MLP(self.max_pool(x))
        return x * self.sigmoid(avg + max)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        y = torch.cat([avg_out, max_out], dim=1)
        return x * self.sigmoid(self.conv(y))

class CBAMBlock(nn.Module):
    def __init__(self, channels, ratio=8, kernel_size=7):
        super().__init__()
        self.ca = ChannelAttention(channels, ratio)
        self.sa = SpatialAttention(kernel_size)

    def forward(self, x):
        x = self.ca(x)
        x = self.sa(x)
        return x

# ✅ 4. 自定义 ViT 模型结构（集成 CBAM + ECA）
class CustomViT_CBAM_ECA(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        base = vit_b_16(weights=ViT_B_16_Weights.DEFAULT)
        self.patch_embed = base.conv_proj
        self.cls_token = base.class_token
        self.pos_embed = base.encoder.pos_embedding
        self.pos_drop = base.encoder.dropout
        self.encoder = base.encoder.layers
        self.norm = base.encoder.ln
        self.head = nn.Linear(base.heads.head.in_features, num_classes)

        self.cbam = CBAMBlock(768)
        self.eca = ECAAttention(768)

    def forward(self, x):
        B = x.shape[0]
        x = self.patch_embed(x).flatten(2).transpose(1, 2)

        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        x = self.pos_drop(x)

        for blk in self.encoder:
            x = blk(x)

        x = self.norm(x)

        feat = x[:, 1:, :].transpose(1, 2).view(B, 768, 14, 14)
        feat = self.cbam(feat)
        feat = self.eca(feat)

        x_cls = x[:, 0]
        out = self.head(x_cls)
        return out

# ✅ 5. 配置路径和参数
train_dir = "/root/autodl-fs/isic19_20_split/train"
val_dir   = "/root/autodl-fs/isic19_20_split/val"
test_dir  = "/root/autodl-fs/isic19_20_split/test"
ckpt_path = "/root/autodl-fs/best_vit.pth"
batch_size = 32
num_epochs = 10
learning_rate = 5e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ 6. 数据加载与预处理
weights = ViT_B_16_Weights.DEFAULT
transform = weights.transforms()
train_dataset = FilteredImageFolder(train_dir, transform=transform)
test_dataset = FilteredImageFolder(test_dir, transform=transform)

if os.path.exists(val_dir):
    val_dataset = FilteredImageFolder(val_dir, transform=transform)
else:
    val_len = int(len(train_dataset) * 0.15)
    train_len = len(train_dataset) - val_len
    train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_len, val_len])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# ✅ 7. 模型训练与验证
num_classes = len(train_dataset.dataset.classes if hasattr(train_dataset, 'dataset') else train_dataset.classes)
model = CustomViT_CBAM_ECA(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

best_val_acc = 0.0
for epoch in range(num_epochs):
    model.train()
    train_loss, train_acc = 0.0, 0
    for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.size(0)
        train_acc += (out.argmax(1) == y).sum().item()

    train_loss /= len(train_loader.dataset)
    train_acc  /= len(train_loader.dataset)

    model.eval()
    val_loss, val_acc = 0.0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            val_loss += loss.item() * x.size(0)
            val_acc  += (out.argmax(1) == y).sum().item()
    val_loss /= len(val_loader.dataset)
    val_acc  /= len(val_loader.dataset)

    print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        os.makedirs(os.path.dirname(ckpt_path), exist_ok=True)
        torch.save(model.state_dict(), ckpt_path)
        print("✅ Saved Best Model!")

    epoch_ckpt_path = f"/root/autodl-fs/ckpt_vit/epoch_{epoch+1}.pth"
    os.makedirs(os.path.dirname(epoch_ckpt_path), exist_ok=True)
    torch.save(model.state_dict(), epoch_ckpt_path)

# ✅ 8. 测试集评估
model.load_state_dict(torch.load(ckpt_path))
model.eval()
test_loss, test_acc, total = 0.0, 0, 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        out = model(x)
        loss = criterion(out, y)
        test_loss += loss.item() * x.size(0)
        test_acc  += (out.argmax(1) == y).sum().item()
        total += y.size(0)

test_loss /= total
test_acc  /= total
print(f"🎯 Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")

# ✅ 9. 所有 Epoch 模型评估
print("\n📊 Evaluating all epoch checkpoints on test set:")
epoch_results = []
for e in range(1, num_epochs + 1):
    ckpt_file = f"/root/autodl-fs/ckpt_vit/epoch_{e}.pth"
    if not os.path.exists(ckpt_file):
        print(f"❌ Epoch {e} model not found.")
        continue

    model.load_state_dict(torch.load(ckpt_file))
    model.eval()
    test_loss, test_acc, total = 0.0, 0, 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            test_loss += loss.item() * x.size(0)
            test_acc  += (out.argmax(1) == y).sum().item()
            total += y.size(0)

    test_loss /= total
    test_acc  /= total
    epoch_results.append((e, test_loss, test_acc))
    print(f"📁 Epoch {e:02d} | Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")


Epoch 1/10:   0%|          | 0/251 [00:00<?, ?it/s]

[Epoch 1] Train Loss: 0.2441, Acc: 0.9012 | Val Loss: 0.1756, Acc: 0.9383
✅ Saved Best Model!


Epoch 2/10:   0%|          | 0/251 [00:00<?, ?it/s]

[Epoch 2] Train Loss: 0.1588, Acc: 0.9396 | Val Loss: 0.1875, Acc: 0.9289


Epoch 3/10:   0%|          | 0/251 [00:00<?, ?it/s]

...

[Epoch 10] Train Loss: 0.0280, Acc: 0.9905 | Val Loss: 0.2976, Acc: 0.9412
✅ Saved Best Model!


/tmp/ipykernel_115943/3576889066.py:187: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_path))


🎯 Test Loss: 0.2881 | Test Accuracy: 0.9395

📊 Evaluating all epoch checkpoints on test set:


/tmp/ipykernel_115943/3576889066.py:212: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_file))


📁 Epoch 01 | Test Loss: 0.1784 | Test Accuracy: 0.9243
📁 Epoch 02 | Test Loss: 0.1747 | Test Accuracy: 0.9336
📁 Epoch 03 | Test Loss: 0.1644 | Test Accuracy: 0.9447
📁 Epoch 04 | Test Loss: 0.1882 | Test Accuracy: 0.9313
📁 Epoch 05 | Test Loss: 0.1948 | Test Accuracy: 0.9348
📁 Epoch 06 | Test Loss: 0.2087 | Test Accuracy: 0.9360
📁 Epoch 07 | Test Loss: 0.2398 | Test Accuracy: 0.9267
📁 Epoch 08 | Test Loss: 0.2563 | Test Accuracy: 0.9377
📁 Epoch 09 | Test Loss: 0.2946 | Test Accuracy: 0.9313
📁 Epoch 10 | Test Loss: 0.2881 | Test Accuracy: 0.9395


In [8]:
# ✅ 1. 导入库
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models.vision_transformer import vit_b_16, ViT_B_16_Weights
from tqdm.notebook import tqdm

# ✅ 2. 忽略隐藏目录的数据集类
class FilteredImageFolder(ImageFolder):
    def find_classes(self, directory):
        classes = [d.name for d in os.scandir(directory) if d.is_dir() and not d.name.startswith('.')]
        classes.sort()
        class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}
        return classes, class_to_idx

# ✅ 3. 定义 CBAM + ECA 模块
class ECAAttention(nn.Module):
    def __init__(self, channels, k_size=3):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv = nn.Conv1d(1, 1, kernel_size=k_size, padding=(k_size-1)//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        y = self.avg_pool(x)
        y = self.conv(y.squeeze(-1).transpose(-1, -2))
        y = self.sigmoid(y).transpose(-1, -2).unsqueeze(-1)
        return x * y.expand_as(x)

class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=8):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.shared_MLP = nn.Sequential(
            nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg = self.shared_MLP(self.avg_pool(x))
        max = self.shared_MLP(self.max_pool(x))
        return x * self.sigmoid(avg + max)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        y = torch.cat([avg_out, max_out], dim=1)
        return x * self.sigmoid(self.conv(y))

class CBAMBlock(nn.Module):
    def __init__(self, channels, ratio=8, kernel_size=7):
        super().__init__()
        self.ca = ChannelAttention(channels, ratio)
        self.sa = SpatialAttention(kernel_size)

    def forward(self, x):
        x = self.ca(x)
        x = self.sa(x)
        return x

# ✅ 4. 自定义 ViT 模型结构（集成 CBAM + ECA）
class CustomViT_CBAM_ECA(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        base = vit_b_16(weights=ViT_B_16_Weights.DEFAULT)
        self.patch_embed = base.conv_proj
        self.cls_token = base.class_token
        self.pos_embed = base.encoder.pos_embedding
        self.pos_drop = base.encoder.dropout
        self.encoder = base.encoder.layers
        self.norm = base.encoder.ln
        self.head = nn.Linear(base.heads.head.in_features, num_classes)

        self.cbam = CBAMBlock(768)
        self.eca = ECAAttention(768)

    def forward(self, x):
        B = x.shape[0]
        x = self.patch_embed(x).flatten(2).transpose(1, 2)

        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        x = self.pos_drop(x)

        for blk in self.encoder:
            x = blk(x)

        x = self.norm(x)

        feat = x[:, 1:, :].transpose(1, 2).view(B, 768, 14, 14)
        feat = self.cbam(feat)
        feat = self.eca(feat)

        x_cls = x[:, 0]
        out = self.head(x_cls)
        return out

# ✅ 5. 配置路径和参数
train_dir = "/root/autodl-fs/processed/processed/train"
val_dir   = "/root/autodl-fs/processed/processed/val"
test_dir  = "/root/autodl-fs/processed/processed/test"
ckpt_path = "/root/autodl-fs/best_vit.pth"
batch_size = 32
num_epochs = 10
learning_rate = 5e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ 6. 数据加载与预处理
weights = ViT_B_16_Weights.DEFAULT
transform = weights.transforms()
train_dataset = FilteredImageFolder(train_dir, transform=transform)
test_dataset = FilteredImageFolder(test_dir, transform=transform)

if os.path.exists(val_dir):
    val_dataset = FilteredImageFolder(val_dir, transform=transform)
else:
    val_len = int(len(train_dataset) * 0.15)
    train_len = len(train_dataset) - val_len
    train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_len, val_len])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# ✅ 7. 模型训练与验证
num_classes = len(train_dataset.dataset.classes if hasattr(train_dataset, 'dataset') else train_dataset.classes)
model = CustomViT_CBAM_ECA(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

best_val_acc = 0.0
for epoch in range(num_epochs):
    model.train()
    train_loss, train_acc = 0.0, 0
    for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.size(0)
        train_acc += (out.argmax(1) == y).sum().item()

    train_loss /= len(train_loader.dataset)
    train_acc  /= len(train_loader.dataset)

    model.eval()
    val_loss, val_acc = 0.0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            val_loss += loss.item() * x.size(0)
            val_acc  += (out.argmax(1) == y).sum().item()
    val_loss /= len(val_loader.dataset)
    val_acc  /= len(val_loader.dataset)

    print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        os.makedirs(os.path.dirname(ckpt_path), exist_ok=True)
        torch.save(model.state_dict(), ckpt_path)
        print("✅ Saved Best Model!")

    epoch_ckpt_path = f"/root/autodl-fs/ckpt_vit/epoch_{epoch+1}.pth"
    os.makedirs(os.path.dirname(epoch_ckpt_path), exist_ok=True)
    torch.save(model.state_dict(), epoch_ckpt_path)

# ✅ 8. 测试集评估
model.load_state_dict(torch.load(ckpt_path))
model.eval()
test_loss, test_acc, total = 0.0, 0, 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        out = model(x)
        loss = criterion(out, y)
        test_loss += loss.item() * x.size(0)
        test_acc  += (out.argmax(1) == y).sum().item()
        total += y.size(0)

test_loss /= total
test_acc  /= total
print(f"🎯 Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")

# ✅ 9. 所有 Epoch 模型评估
print("\n📊 Evaluating all epoch checkpoints on test set:")
epoch_results = []
for e in range(1, num_epochs + 1):
    ckpt_file = f"/root/autodl-fs/ckpt_vit/epoch_{e}.pth"
    if not os.path.exists(ckpt_file):
        print(f"❌ Epoch {e} model not found.")
        continue

    model.load_state_dict(torch.load(ckpt_file))
    model.eval()
    test_loss, test_acc, total = 0.0, 0, 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            test_loss += loss.item() * x.size(0)
            test_acc  += (out.argmax(1) == y).sum().item()
            total += y.size(0)

    test_loss /= total
    test_acc  /= total
    epoch_results.append((e, test_loss, test_acc))
    print(f"📁 Epoch {e:02d} | Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")


Epoch 1/10:   0%|          | 0/287 [00:00<?, ?it/s]

[Epoch 1] Train Loss: 0.2309, Acc: 0.9068 | Val Loss: 0.1923, Acc: 0.9238
✅ Saved Best Model!


Epoch 2/10:   0%|          | 0/287 [00:00<?, ?it/s]

[Epoch 2] Train Loss: 0.1599, Acc: 0.9407 | Val Loss: 0.3334, Acc: 0.8737


Epoch 3/10:   0%|          | 0/287 [00:00<?, ?it/s]

...

[Epoch 10] Train Loss: 0.0181, Acc: 0.9939 | Val Loss: 0.3143, Acc: 0.9158


/tmp/ipykernel_115943/4140671554.py:187: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_path))


🎯 Test Loss: 0.4335 | Test Accuracy: 0.8960

📊 Evaluating all epoch checkpoints on test set:


/tmp/ipykernel_115943/4140671554.py:212: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_file))


📁 Epoch 01 | Test Loss: 0.2698 | Test Accuracy: 0.8843
📁 Epoch 02 | Test Loss: 0.4548 | Test Accuracy: 0.8243
📁 Epoch 03 | Test Loss: 0.2467 | Test Accuracy: 0.9165
📁 Epoch 04 | Test Loss: 0.2628 | Test Accuracy: 0.9151
📁 Epoch 05 | Test Loss: 0.2986 | Test Accuracy: 0.9107
📁 Epoch 06 | Test Loss: 0.4499 | Test Accuracy: 0.8931
📁 Epoch 07 | Test Loss: 0.3970 | Test Accuracy: 0.8990
📁 Epoch 08 | Test Loss: 0.4238 | Test Accuracy: 0.8960
📁 Epoch 09 | Test Loss: 0.4335 | Test Accuracy: 0.8960
📁 Epoch 10 | Test Loss: 0.4339 | Test Accuracy: 0.8829


In [9]:
# ✅ 1. 导入库
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models.vision_transformer import vit_b_16, ViT_B_16_Weights
from tqdm.notebook import tqdm

# ✅ 2. 忽略隐藏目录的数据集类
class FilteredImageFolder(ImageFolder):
    def find_classes(self, directory):
        classes = [d.name for d in os.scandir(directory) if d.is_dir() and not d.name.startswith('.')]
        classes.sort()
        class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}
        return classes, class_to_idx

# ✅ 3. 定义 CBAM + ECA 模块
class ECAAttention(nn.Module):
    def __init__(self, channels, k_size=3):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv = nn.Conv1d(1, 1, kernel_size=k_size, padding=(k_size-1)//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        y = self.avg_pool(x)
        y = self.conv(y.squeeze(-1).transpose(-1, -2))
        y = self.sigmoid(y).transpose(-1, -2).unsqueeze(-1)
        return x * y.expand_as(x)

class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=8):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.shared_MLP = nn.Sequential(
            nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg = self.shared_MLP(self.avg_pool(x))
        max = self.shared_MLP(self.max_pool(x))
        return x * self.sigmoid(avg + max)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        y = torch.cat([avg_out, max_out], dim=1)
        return x * self.sigmoid(self.conv(y))

class CBAMBlock(nn.Module):
    def __init__(self, channels, ratio=8, kernel_size=7):
        super().__init__()
        self.ca = ChannelAttention(channels, ratio)
        self.sa = SpatialAttention(kernel_size)

    def forward(self, x):
        x = self.ca(x)
        x = self.sa(x)
        return x

# ✅ 4. 自定义 ViT 模型结构（集成 CBAM + ECA）
class CustomViT_CBAM_ECA(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        base = vit_b_16(weights=ViT_B_16_Weights.DEFAULT)
        self.patch_embed = base.conv_proj
        self.cls_token = base.class_token
        self.pos_embed = base.encoder.pos_embedding
        self.pos_drop = base.encoder.dropout
        self.encoder = base.encoder.layers
        self.norm = base.encoder.ln
        self.head = nn.Linear(base.heads.head.in_features, num_classes)

        self.cbam = CBAMBlock(768)
        self.eca = ECAAttention(768)

    def forward(self, x):
        B = x.shape[0]
        x = self.patch_embed(x).flatten(2).transpose(1, 2)

        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        x = self.pos_drop(x)

        for blk in self.encoder:
            x = blk(x)

        x = self.norm(x)

        feat = x[:, 1:, :].transpose(1, 2).view(B, 768, 14, 14)
        feat = self.cbam(feat)
        feat = self.eca(feat)

        x_cls = x[:, 0]
        out = self.head(x_cls)
        return out

# ✅ 5. 配置路径和参数
train_dir = "/root/autodl-fs/generate/train"
val_dir   = "/root/autodl-fs/generate/val"
test_dir  = "/root/autodl-fs/generate/test"
ckpt_path = "/root/autodl-fs/best_vit.pth"
batch_size = 32
num_epochs = 10
learning_rate = 5e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ 6. 数据加载与预处理
weights = ViT_B_16_Weights.DEFAULT
transform = weights.transforms()
train_dataset = FilteredImageFolder(train_dir, transform=transform)
test_dataset = FilteredImageFolder(test_dir, transform=transform)

if os.path.exists(val_dir):
    val_dataset = FilteredImageFolder(val_dir, transform=transform)
else:
    val_len = int(len(train_dataset) * 0.15)
    train_len = len(train_dataset) - val_len
    train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_len, val_len])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# ✅ 7. 模型训练与验证
num_classes = len(train_dataset.dataset.classes if hasattr(train_dataset, 'dataset') else train_dataset.classes)
model = CustomViT_CBAM_ECA(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

best_val_acc = 0.0
for epoch in range(num_epochs):
    model.train()
    train_loss, train_acc = 0.0, 0
    for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.size(0)
        train_acc += (out.argmax(1) == y).sum().item()

    train_loss /= len(train_loader.dataset)
    train_acc  /= len(train_loader.dataset)

    model.eval()
    val_loss, val_acc = 0.0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            val_loss += loss.item() * x.size(0)
            val_acc  += (out.argmax(1) == y).sum().item()
    val_loss /= len(val_loader.dataset)
    val_acc  /= len(val_loader.dataset)

    print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        os.makedirs(os.path.dirname(ckpt_path), exist_ok=True)
        torch.save(model.state_dict(), ckpt_path)
        print("✅ Saved Best Model!")

    epoch_ckpt_path = f"/root/autodl-fs/ckpt_vit/epoch_{epoch+1}.pth"
    os.makedirs(os.path.dirname(epoch_ckpt_path), exist_ok=True)
    torch.save(model.state_dict(), epoch_ckpt_path)

# ✅ 8. 测试集评估
model.load_state_dict(torch.load(ckpt_path))
model.eval()
test_loss, test_acc, total = 0.0, 0, 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        out = model(x)
        loss = criterion(out, y)
        test_loss += loss.item() * x.size(0)
        test_acc  += (out.argmax(1) == y).sum().item()
        total += y.size(0)

test_loss /= total
test_acc  /= total
print(f"🎯 Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")

# ✅ 9. 所有 Epoch 模型评估
print("\n📊 Evaluating all epoch checkpoints on test set:")
epoch_results = []
for e in range(1, num_epochs + 1):
    ckpt_file = f"/root/autodl-fs/ckpt_vit/epoch_{e}.pth"
    if not os.path.exists(ckpt_file):
        print(f"❌ Epoch {e} model not found.")
        continue

    model.load_state_dict(torch.load(ckpt_file))
    model.eval()
    test_loss, test_acc, total = 0.0, 0, 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            test_loss += loss.item() * x.size(0)
            test_acc  += (out.argmax(1) == y).sum().item()
            total += y.size(0)

    test_loss /= total
    test_acc  /= total
    epoch_results.append((e, test_loss, test_acc))
    print(f"📁 Epoch {e:02d} | Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")


Epoch 1/10:   0%|          | 0/286 [00:00<?, ?it/s]

[Epoch 1] Train Loss: 0.2416, Acc: 0.9002 | Val Loss: 0.1787, Acc: 0.9344
✅ Saved Best Model!


Epoch 2/10:   0%|          | 0/286 [00:00<?, ?it/s]

[Epoch 2] Train Loss: 0.1686, Acc: 0.9352 | Val Loss: 0.1501, Acc: 0.9455
✅ Saved Best Model!


Epoch 3/10:   0%|          | 0/286 [00:00<?, ?it/s]

...

[Epoch 10] Train Loss: 0.0290, Acc: 0.9893 | Val Loss: 0.2760, Acc: 0.9424


/tmp/ipykernel_115943/3880948063.py:187: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_path))


🎯 Test Loss: 0.2297 | Test Accuracy: 0.9253

📊 Evaluating all epoch checkpoints on test set:


/tmp/ipykernel_115943/3880948063.py:212: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_file))


📁 Epoch 01 | Test Loss: 0.2519 | Test Accuracy: 0.9195
📁 Epoch 02 | Test Loss: 0.2297 | Test Accuracy: 0.9253
📁 Epoch 03 | Test Loss: 0.2720 | Test Accuracy: 0.9078
📁 Epoch 04 | Test Loss: 0.2507 | Test Accuracy: 0.9224
📁 Epoch 05 | Test Loss: 0.3472 | Test Accuracy: 0.8990
📁 Epoch 06 | Test Loss: 0.4782 | Test Accuracy: 0.8858
📁 Epoch 07 | Test Loss: 0.2603 | Test Accuracy: 0.9136
📁 Epoch 08 | Test Loss: 0.3238 | Test Accuracy: 0.9239
📁 Epoch 09 | Test Loss: 0.3764 | Test Accuracy: 0.9048
📁 Epoch 10 | Test Loss: 0.5640 | Test Accuracy: 0.8873


In [10]:
# ✅ 1. 导入库
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models.vision_transformer import vit_b_16, ViT_B_16_Weights
from tqdm.notebook import tqdm

# ✅ 2. 忽略隐藏目录的数据集类
class FilteredImageFolder(ImageFolder):
    def find_classes(self, directory):
        classes = [d.name for d in os.scandir(directory) if d.is_dir() and not d.name.startswith('.')]
        classes.sort()
        class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}
        return classes, class_to_idx

# ✅ 3. 定义 CBAM + ECA 模块
class ECAAttention(nn.Module):
    def __init__(self, channels, k_size=3):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv = nn.Conv1d(1, 1, kernel_size=k_size, padding=(k_size-1)//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        y = self.avg_pool(x)
        y = self.conv(y.squeeze(-1).transpose(-1, -2))
        y = self.sigmoid(y).transpose(-1, -2).unsqueeze(-1)
        return x * y.expand_as(x)

class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=8):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.shared_MLP = nn.Sequential(
            nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg = self.shared_MLP(self.avg_pool(x))
        max = self.shared_MLP(self.max_pool(x))
        return x * self.sigmoid(avg + max)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        y = torch.cat([avg_out, max_out], dim=1)
        return x * self.sigmoid(self.conv(y))

class CBAMBlock(nn.Module):
    def __init__(self, channels, ratio=8, kernel_size=7):
        super().__init__()
        self.ca = ChannelAttention(channels, ratio)
        self.sa = SpatialAttention(kernel_size)

    def forward(self, x):
        x = self.ca(x)
        x = self.sa(x)
        return x

# ✅ 4. 自定义 ViT 模型结构（集成 CBAM + ECA）
class CustomViT_CBAM_ECA(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        base = vit_b_16(weights=ViT_B_16_Weights.DEFAULT)
        self.patch_embed = base.conv_proj
        self.cls_token = base.class_token
        self.pos_embed = base.encoder.pos_embedding
        self.pos_drop = base.encoder.dropout
        self.encoder = base.encoder.layers
        self.norm = base.encoder.ln
        self.head = nn.Linear(base.heads.head.in_features, num_classes)

        self.cbam = CBAMBlock(768)
        self.eca = ECAAttention(768)

    def forward(self, x):
        B = x.shape[0]
        x = self.patch_embed(x).flatten(2).transpose(1, 2)

        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        x = self.pos_drop(x)

        for blk in self.encoder:
            x = blk(x)

        x = self.norm(x)

        feat = x[:, 1:, :].transpose(1, 2).view(B, 768, 14, 14)
        feat = self.cbam(feat)
        feat = self.eca(feat)

        x_cls = x[:, 0]
        out = self.head(x_cls)
        return out

# ✅ 5. 配置路径和参数
train_dir = "/root/autodl-fs/generate_twice/train"
val_dir   = "/root/autodl-fs/generate_twice/val"
test_dir  = "/root/autodl-fs/generate_twice/test"
ckpt_path = "/root/autodl-fs/best_vit.pth"
batch_size = 32
num_epochs = 10
learning_rate = 5e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ 6. 数据加载与预处理
weights = ViT_B_16_Weights.DEFAULT
transform = weights.transforms()
train_dataset = FilteredImageFolder(train_dir, transform=transform)
test_dataset = FilteredImageFolder(test_dir, transform=transform)

if os.path.exists(val_dir):
    val_dataset = FilteredImageFolder(val_dir, transform=transform)
else:
    val_len = int(len(train_dataset) * 0.15)
    train_len = len(train_dataset) - val_len
    train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_len, val_len])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# ✅ 7. 模型训练与验证
num_classes = len(train_dataset.dataset.classes if hasattr(train_dataset, 'dataset') else train_dataset.classes)
model = CustomViT_CBAM_ECA(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

best_val_acc = 0.0
for epoch in range(num_epochs):
    model.train()
    train_loss, train_acc = 0.0, 0
    for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x.size(0)
        train_acc += (out.argmax(1) == y).sum().item()

    train_loss /= len(train_loader.dataset)
    train_acc  /= len(train_loader.dataset)

    model.eval()
    val_loss, val_acc = 0.0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            val_loss += loss.item() * x.size(0)
            val_acc  += (out.argmax(1) == y).sum().item()
    val_loss /= len(val_loader.dataset)
    val_acc  /= len(val_loader.dataset)

    print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        os.makedirs(os.path.dirname(ckpt_path), exist_ok=True)
        torch.save(model.state_dict(), ckpt_path)
        print("✅ Saved Best Model!")

    epoch_ckpt_path = f"/root/autodl-fs/ckpt_vit/epoch_{epoch+1}.pth"
    os.makedirs(os.path.dirname(epoch_ckpt_path), exist_ok=True)
    torch.save(model.state_dict(), epoch_ckpt_path)

# ✅ 8. 测试集评估
model.load_state_dict(torch.load(ckpt_path))
model.eval()
test_loss, test_acc, total = 0.0, 0, 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        out = model(x)
        loss = criterion(out, y)
        test_loss += loss.item() * x.size(0)
        test_acc  += (out.argmax(1) == y).sum().item()
        total += y.size(0)

test_loss /= total
test_acc  /= total
print(f"🎯 Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")

# ✅ 9. 所有 Epoch 模型评估
print("\n📊 Evaluating all epoch checkpoints on test set:")
epoch_results = []
for e in range(1, num_epochs + 1):
    ckpt_file = f"/root/autodl-fs/ckpt_vit/epoch_{e}.pth"
    if not os.path.exists(ckpt_file):
        print(f"❌ Epoch {e} model not found.")
        continue

    model.load_state_dict(torch.load(ckpt_file))
    model.eval()
    test_loss, test_acc, total = 0.0, 0, 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            test_loss += loss.item() * x.size(0)
            test_acc  += (out.argmax(1) == y).sum().item()
            total += y.size(0)

    test_loss /= total
    test_acc  /= total
    epoch_results.append((e, test_loss, test_acc))
    print(f"📁 Epoch {e:02d} | Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")


Epoch 1/10:   0%|          | 0/283 [00:00<?, ?it/s]

[Epoch 1] Train Loss: 0.3028, Acc: 0.8683 | Val Loss: 0.2248, Acc: 0.9110
✅ Saved Best Model!


Epoch 2/10:   0%|          | 0/283 [00:00<?, ?it/s]

[Epoch 2] Train Loss: 0.2276, Acc: 0.9034 | Val Loss: 0.2269, Acc: 0.9179
✅ Saved Best Model!


Epoch 3/10:   0%|          | 0/283 [00:00<?, ?it/s]

...

[Epoch 10] Train Loss: 0.0298, Acc: 0.9888 | Val Loss: 0.4152, Acc: 0.9054


/tmp/ipykernel_115943/150587572.py:187: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_path))


🎯 Test Loss: 0.2906 | Test Accuracy: 0.8949

📊 Evaluating all epoch checkpoints on test set:


/tmp/ipykernel_115943/150587572.py:212: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_file))


📁 Epoch 01 | Test Loss: 0.2799 | Test Accuracy: 0.8715
📁 Epoch 02 | Test Loss: 0.2730 | Test Accuracy: 0.9022
📁 Epoch 03 | Test Loss: 0.2984 | Test Accuracy: 0.8847
📁 Epoch 04 | Test Loss: 0.2906 | Test Accuracy: 0.8949
📁 Epoch 05 | Test Loss: 0.3873 | Test Accuracy: 0.8993
📁 Epoch 06 | Test Loss: 0.6500 | Test Accuracy: 0.8234
📁 Epoch 07 | Test Loss: 0.4037 | Test Accuracy: 0.8978
📁 Epoch 08 | Test Loss: 0.4825 | Test Accuracy: 0.8861
📁 Epoch 09 | Test Loss: 0.4430 | Test Accuracy: 0.9022
📁 Epoch 10 | Test Loss: 0.5337 | Test Accuracy: 0.8949
